# HW4
- 開始寫作業前，若使用 Colab 請先設定使用 GPU!!
- 資料夾結構
```
📦 HW4
┣ helper_funcs.py
┣ main_hw4.ipynb
┣ model.py
┣ requirements.txt
```

In [1]:
# 0. 安裝套件

!pip install torch==2.6.0 --index-url https://download.pytorch.org/whl/cu124
# 安裝requirement中的套件
!pip install -r requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia

In [2]:
# 1. 載入套件

from tqdm import tqdm # progress bar
# PyTorch 系列
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

from medmnist import PneumoniaMNIST # 本作業資料集

from transformers import AutoModelForImageClassification
from transformers import AutoImageProcessor
from transformers import ViTModel

# Hugging Face PEFT
from peft import LoraConfig, get_peft_model

# Custom modules
from helper_funcs import do_test, load_HF_model
from model import CustomViTClassifier

In [3]:
# 2. 超參數

BATCH_SIZE = 32 # 每個batch中的資料數量
EPOCHS = 3 # 訓練三次
LEARNING_RATE = 1e-4 # 每次參數更新的幅度
IMAGE_SIZE = 128 # 影像的尺寸
PATCH_SIZE = 16 # ViT 模型中影像分割成的圖塊
#預訓練模型
MODEL_NAME = "google/vit-base-patch16-224-in21k" # or google/vit-base-patch16-224-in21k google/vit-base-patch16-224
MODEL_TYPE = "HF" # 使用的模型類別
# 檢查 MODEL_TYPE 是否為 "HF" 或 "custom"不然會出現錯誤
assert MODEL_TYPE in ["HF", "custom"], "IMPLEMENTATION must be either 'HF' or 'custom'"
SAVE_NAME = f"lora-vit-pneumonia_ep{EPOCHS}"# 儲存訓練好的模型時的檔案名稱

In [4]:
tmp = PneumoniaMNIST(split="test", download=True)# 創建一個測試資料集

100%|██████████| 4.17M/4.17M [00:01<00:00, 3.42MB/s]


In [5]:
print(tmp) #輸出資料集

Dataset PneumoniaMNIST of size 28 (pneumoniamnist)
    Number of datapoints: 624
    Root location: /root/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'normal', '1': 'pneumonia'}
    Number of samples: {'train': 4708, 'val': 524, 'test': 624}
    Description: The PneumoniaMNIST is based on a prior dataset of 5,856 pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.
    License: CC BY 4.0


In [6]:
# 3. 影像資料前處理準備
# 從預訓練模型載入影像處理器，進行前處理，do_rescale=False不進行影像縮放
image_processor = AutoImageProcessor.from_pretrained(MODEL_NAME, do_rescale=False)
# 使用 image_processor 提供的平均值與標準差,建立 Normalize
normalize = Normalize(
    mean=[image_processor.image_mean[0]],
    std=[image_processor.image_std[0]],
)
# Data Transform
# 使用 transforms.Compose 將多個影像組合成一個序列
transform = transforms.Compose(
    [   # 將影像調整至正確尺寸
        transforms.Resize((image_processor.size["height"], image_processor.size["width"])),
        # 轉換成tensor
        transforms.ToTensor(),
        # 進行標準化
        normalize, # 可以改成你喜歡的
    ]
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [7]:
# 4. Load PneumoniaMNIST

# 載入train資料集,並將前面定義的影像轉換應用到每個從資料集中載入的影像上。
train_raw = PneumoniaMNIST(split="train", download=True, transform=transform)
# 載入test資料集,並將前面定義的影像轉換應用到每個從資料集中載入的影像上。
test_raw = PneumoniaMNIST(split="test", download=True, transform=transform)
# 將原始的訓練集 (train_raw) 分割成訓練集 (train_data) 和驗證集 (val_data)
# 80%的資料用來訓練,20%用來驗證
train_data, val_data = torch.utils.data.random_split(train_raw, [0.8, 0.2])

In [8]:
print(type(tmp))
tmp[1][0]

<class 'medmnist.dataset.PneumoniaMNIST'>


In [9]:
# 5. 創建 DataLoader (TODO1)
# 創建資料集用指定的batch size並在入剛剛切分的資料
# train打亂資料順序用於分類,評估和測試集則不用因為避免結果不准
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)#  Write your code here)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)#  Write your code here)
test_loader = DataLoader(test_raw, batch_size=BATCH_SIZE, shuffle=False)# Write your code here)

In [10]:
# 6. 載入 ViT 模型

#決定載入哪種類型的模型
if MODEL_TYPE == "HF":
    # Load ViT Model
    model = load_HF_model(MODEL_NAME, num_labels=2)
elif MODEL_TYPE == "custom":
    base_model = ViTModel.from_pretrained(MODEL_NAME)
    model = CustomViTClassifier(base_model, num_labels=2)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 7. 設定 LoRA 參數

lora_config = LoraConfig(
    # task_type=TaskType.IMAGE_CLASSIFICATION, # ViT is a (image) sequence classification model
    r=8, # 修改值8,24,控制 LoRA 矩陣的維度、大小
    lora_alpha=16, # 修改值16,48,調整LoRA的縮放
    lora_dropout=0.1, #Dropout防止過度擬合
    # 自注意力中的 query、key、value
    target_modules=["query", "key", "value"], # https://github.com/huggingface/peft/issues/2128
    bias="none", # 是否包含偏置項
    #指定除了 LoRA 模組外，還要保留哪些模組參與訓練，這裡是保留 classifier 層
    modules_to_save=["classifier"],
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# 印出可訓練參數的數量
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"可訓練參數數量: {trainable_params} ({trainable_params/total_params:.2%})")

可訓練參數數量: 443906 (0.51%)


In [12]:
# 8. 訓練前準備
# 使用Adam當optimizer並用先前定義的學習率控制更新幅度
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# 使用CrossEntropyLoss計算損失函數
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
# 如果有 GPU 則使用 'cuda'，否則使用 CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 將模型移動到對應的運算裝置上（GPU 或 CPU）,訓練過程中模型與資料會在同一裝置上運算
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): ViTForImageClassification(
      (vit): ViTModel(
        (embeddings): ViTEmbeddings(
          (patch_embeddings): ViTPatchEmbeddings(
            (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): ViTEncoder(
          (layer): ModuleList(
            (0-11): 12 x ViTLayer(
              (attention): ViTAttention(
                (attention): ViTSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (

In [13]:
# 9. PyTorch training loop
# 遍歷每個訓練 epoch
for epoch in range(EPOCHS):
    # 使用tqdm 讓他在訓練時顯示進度條。
    pbar = tqdm(train_loader)
    # 設定進度條的描述
    pbar.set_description(f"Training epoch [{epoch+1}/{EPOCHS}]")
    model.train()
    # 遍歷訓練資料集中的每個batch
    for pixel_values, labels in pbar:
      #因為預訓練模型要輸入三個通道
        pixel_values = pixel_values.repeat(1, 3, 1, 1)  # 複製通道 3 次

        # TODO2-1
        # A part of the training script is missing.
        # Write your code here

        # 將數據移動到之前指定的cpu或gpu
        pixel_values, labels = pixel_values.to(device), labels.to(device)
        #將優化器的梯度歸0
        optimizer.zero_grad()
        # 將影像數據傳入對應模型
        if MODEL_TYPE == "HF":
            outputs = model(pixel_values=pixel_values, labels=labels).logits
        elif MODEL_TYPE == "custom":
            outputs = model(pixel_values=pixel_values)
        labels = labels.squeeze() # 移除批次維度
        loss = loss_fn(outputs, labels) # 計算loss



        # TODO2-2
        # A part of the training script is missing.
        # Write your code here
        loss.backward() # 進行反向傳播
        optimizer.step() # 更新模型的權重
        # 在進度條上顯示當前的損失值
        pbar.set_postfix(loss = loss.item())
    # 在每個 epoch 結束後印出最終損失值
    print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {loss.item():.4f}')
    #在驗證集上評估模型
    do_test(val_loader, model, MODEL_TYPE, loss_fn, device, EPOCHS, epoch)

Training epoch [1/3]: 100%|██████████| 118/118 [01:32<00:00,  1.28it/s, loss=0.156]


Epoch 1/3, Loss: 0.1560


validation epoch [1/3]: 100%|██████████| 30/30 [00:12<00:00,  2.40it/s]


Accuracy: 0.9086 
F1 Score: 0.8748


Training epoch [2/3]: 100%|██████████| 118/118 [01:39<00:00,  1.18it/s, loss=0.0844]


...

Accuracy: 0.9575 
F1 Score: 0.9448


Training epoch [3/3]: 100%|██████████| 118/118 [01:38<00:00,  1.19it/s, loss=0.238]


Epoch 3/3, Loss: 0.2385


validation epoch [3/3]: 100%|██████████| 30/30 [00:11<00:00,  2.54it/s]

Accuracy: 0.9607 
F1 Score: 0.9488


In [14]:
# 10. test the trained model
# 在測試集評估訓練好的模型
do_test(test_loader, model, MODEL_TYPE, loss_fn, device, EPOCHS, epoch, "test")


test epoch [3/3]: 100%|██████████| 20/20 [00:07<00:00,  2.57it/s]

Accuracy: 0.8301 
F1 Score: 0.7964


0.5298087134957313